# Assignment 1-1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to download HTML pages from a website?
* How to extract relevant content from an HTML page? 

Furthermore, you will gain a deeper understanding of the data science lifecycle.

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) rather than [lxml](http://lxml.de/) to parse an HTML page and extract data from the page.

3. Please follow the python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of this topic. I found that this is a good resource: [Tutorial: Web Scraping and BeautifulSoup](https://www.dataquest.io/blog/web-scraping-beautifulsoup/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. Your job is to extract insights from SFU data to answer questions. 

In this assignment, you will do two tasks. Please recall the high-level data science lifecycle from Lecture 1. I suggest that when doing this assignment, please remind yourself of what data you collected and what questions you tried to answer.

## Task 1: SFU CS Faculty Members

Sometimes you don't know what questions to ask. No worries. Start collecting data first. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [http://www.sfu.ca/computing/people/faculty.html](http://www.sfu.ca/computing/people/faculty.html).




### (a) Crawl Web Page

A web page is essentially a file stored in a remote machine (called web server). Please write code to download the HTML page and save it as a text file ("csfaculty.html").

In [7]:
# write your code
import requests

def download_html(url, filename):
    try:
        # Send an HTTP request to the specified URL
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Save the HTML content to a text file
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(response.text)
            print(f"HTML page downloaded and saved as {filename}")
        else:
            print(f"Failed to download HTML. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Specify the URL of the HTML page you want to download
url = "http://www.sfu.ca/computing/people/faculty.html"

# Specify the desired filename for the saved HTML content
filename = "csfaculty.html"

# Call the function to download and save the HTML page
download_html(url, filename)

HTML page downloaded and saved as csfaculty.html


### (b) Extract Structured Data

Please write code to extract relevant content (name, rank, area, profile, homepage) from "csfaculty.html" and save them as a CSV file (like [faculty_table.csv](./faculty_table.csv)). 

In [8]:
# write your code
from bs4 import BeautifulSoup
import csv

def extract_faculty_data(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    faculty_data = []
    faculty_div = soup.find('div', class_='text')
    
    while True:
       h4_tag = faculty_div.find('h4')
       if h4_tag:
         name_and_rank = h4_tag.text.strip().split(', ')
         name = name_and_rank[0]
         rank_long = name_and_rank[1] if len(name_and_rank) > 1 else None
         if rank_long is not None:
           if "Assistant Professor" in rank_long or "ASSISTANT PROFESSOR" in rank_long:
                rank = "Assistant Professor"
           elif "Associate Professor" in rank_long   or "ASSOCIATE PROFESSOR" in rank_long:
                rank = "Associate Professor"
           elif "Professor" in rank_long  or "PROFESSOR" in rank_long:
               rank = "Professor"
           elif "Senior Lecturer" in rank_long  or "SENIOR LECTURER" in rank_long:
               rank = "Senior Lecturer"
           elif "Lecturer" in rank_long  or "LECTURER" in rank_long:
               rank = "Lecturer"
           else: rank = "other professor title"
         else: rank = "other professor title"
       
         # Extracting area
         area_flag = faculty_div.find('b', text=['Area:', 'Area: ','Area','Area:&nbsp;','Area','Area:&nbsp; '])
        
         if area_flag is not None:
            area = area_flag.next_sibling.get_text(strip=True)            
         else: area = None

         # Extracting profile and homepage
         profile_link = faculty_div.find('a', text='Profile & Contact Information')
         if profile_link is not None:
            profile = profile_link.get('href', None)
         else:
            profile = None
         
         homepage_link = faculty_div.find('a', text='Home Page')
         if homepage_link is not None:
            homepage = homepage_link.get('href', None)
         else:
            homepage = None
         faculty_data.append([name, rank, area, profile, homepage])
       faculty_div = faculty_div.find_next('div', class_='text') if faculty_div else None
       if faculty_div is None:
           break
       else:
           name = None
           rank = None
           area = None 
           profile = None
           homepage =None
        
    return faculty_data

def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        header = ["Name", "Rank", "Area", "Profile", "Homepage"]
        csv_writer.writerow(header)
        csv_writer.writerows(data)

if __name__ == "__main__":
    # Load the HTML content from the file
    with open("csfaculty.html", 'r', encoding='utf-8') as html_file:
        html_content = html_file.read()

    # Extract faculty data
    faculty_data = extract_faculty_data(html_content)

    # Save the data to a CSV file
    save_to_csv(faculty_data, "faculty_table.csv")

    print("CSV file saved successfully.")

CSV file saved successfully.


C:\Users\duanq\AppData\Local\Temp\ipykernel_18100\2396079795.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  area_flag = faculty_div.find('b', text=['Area:', 'Area: ','Area','Area:&nbsp;','Area','Area:&nbsp; '])
C:\Users\duanq\AppData\Local\Temp\ipykernel_18100\2396079795.py:39: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  profile_link = faculty_div.find('a', text='Profile & Contact Information')
C:\Users\duanq\AppData\Local\Temp\ipykernel_18100\2396079795.py:45: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  homepage_link = faculty_div.find('a', text='Home Page')


### (c) Interesting Finding

Note that you don't need to do anything for Task 1(c). The purpose of this part is to give you some sense about how to leverage exploratory data analysis (EDA) to come up with interesting questions about the data. EDA is an important topic in data science; you will  learn it soon from this course. 


First, please install [dataprep](http://dataprep.ai).
Then, run the cell below. 
It shows a bar chart for every column. What interesting findings can you get from these visualizations? 

In [ ]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_table.csv")
plot(df)

Below are some examples:

**Finding 1:** Professor# (26) is more than 2x larger than Associate Professor# (10). 

**Questions:** Why did it happen? Is it common in all CS schools in Canada? Will the gap go larger or smaller in five years? What actions can be taken to enlarge/shrink the gap?


**Finding 2:** The Homepage has 22% missing values. 

**Questions:** Why are there so many missing values? Is it because many faculty do not have their own homepages or do not add their homepages to the school page? What actions can be taken to avoid this to happen in the future? 

## Task 2: Age Follows Normal Distribution?

In this task, you start with a question and then figure out what data to collect.

The question that you are interested in is `Does SFU CS faculty age follow a normal distribution?`

To estimate the age of a faculty member, you can collect the year in which s/he graduates from a university (`gradyear`) and then estimate `age` using the following equation:

$$age \approx 2021+23 - gradyear$$

For example, if one graduates from a university in 1990, then the age is estimated as 2021+23-1990 = 54. 



### (a) Crawl Web Page

You notice that faculty profile pages contain graduation information. For example, you can see that Dr. Jiannan Wang graduated from Harbin Institute of Technology in 2008 at [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please write code to download the 68 profile pages and save each page as a text file. 

In [ ]:
# Write your code

### (b) Extract Structured Data

Please write code to extract the earliest graduation year (e.g., 2008 for Dr. Jiannan Wang) from each profile page, and create a csv file like [faculty_grad_year.csv](./faculty_grad_year.csv). 

In [ ]:
# write your code here

### (c) Interesting Finding

Similar to Task 1(c), you don't need to do anything here. Just look at different visualizations w.r.t. age and give yourself an answer to the question: `Does SFU CS faculty age follow a normal distribution?`

In [ ]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_grad_year.csv")
df["age"] = 2021+23-df["gradyear"]

plot(df, "age")

## Submission

Complete the code in this notebook, and submit it to the CourSys activity `Assignment 1`.